### Heart Disease Dataset

The data came from [Kaggle](https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset?select=heart.csv)

A logistic regression model is used to predict whether someone has heart disease or not, based on a set of features that is related to the presence of heart disease. 

- age
- sex
- cp: chest pain type (4 values)
- trestbps: resting blood pressure
- col: serum cholestoral in mg/dl
- fbs: fasting blood sugar > 120 mg/dl
- restecg: resting electrocardiographic results (values 0,1,2)
- thalach: maximum heart rate achieved
- exang: exercise induced angina
- oldpeak: ST depression induced by exercise relative to rest
- slope: the slope of the peak exercise ST segment
- ca: number of major vessels (0-3) colored by flourosopy
- thal: 0 = normal; 1 = fixed defect; 2 = reversable defect

We can train the logistic regression model on a labeled dataset, where each example is labeled with either 0 (no heart disease) or 1 (heart disease).

Once the model is trained, we can use it to make predictions on new, unseen data. For each new example, the logistic regression model will output a predicted probability of having heart disease, based on the values of the input features. If the predicted probability is greater than a certain threshold, we can classify the example as having heart disease, otherwise we can classify it as not having heart disease.

To evaluate the performance of the logistic regression model, we can use a confusion matrix. A confusion matrix is a table that summarizes the number of true positives, true negatives, false positives, and false negatives produced by the model on a given set of examples. True positives (TP) are the number of examples that were correctly classified as having heart disease, true negatives (TN) are the number of examples that were correctly classified as not having heart disease, false positives (FP) are the number of examples that were incorrectly classified as having heart disease (false alarms), and false negatives (FN) are the number of examples that were incorrectly classified as not having heart disease (missed detections).

By analyzing the confusion matrix, we can calculate metrics such as accuracy, precision, recall, and F1-score, which provide a more detailed assessment of the model's performance. For example, accuracy measures the proportion of examples that were classified correctly, while precision measures the proportion of positive predictions that were correct, and recall measures the proportion of actual positives that were correctly predicted. F1-score is the harmonic mean of precision and recall, and is often used as a summary metric that balances the trade-off between precision and recall.

In [31]:
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
df =pd.read_csv('heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [4]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [5]:
df.shape

(1025, 14)

In [10]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1025.0,54.434146,9.072290,29.0,48.0,56.0,61.0,77.0
sex,1025.0,0.695610,0.460373,0.0,0.0,1.0,1.0,1.0
cp,1025.0,0.942439,1.029641,0.0,0.0,1.0,2.0,3.0
trestbps,1025.0,131.611707,17.516718,94.0,120.0,130.0,140.0,200.0
chol,1025.0,246.000000,51.592510,126.0,211.0,240.0,275.0,564.0
fbs,1025.0,0.149268,0.356527,0.0,0.0,0.0,0.0,1.0
restecg,1025.0,0.529756,0.527878,0.0,0.0,1.0,1.0,2.0
thalach,1025.0,149.114146,23.005724,71.0,132.0,152.0,166.0,202.0
exang,1025.0,0.336585,0.472772,0.0,0.0,0.0,1.0,1.0
oldpeak,1025.0,1.071512,1.175053,0.0,0.0,0.8,1.8,6.2


In [16]:
y = df['target']
x = df.drop('target', axis=1)

In [17]:
x

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2


In [18]:
y

0       0
1       0
2       0
3       0
4       0
       ..
1020    1
1021    0
1022    0
1023    1
1024    0
Name: target, Length: 1025, dtype: int64

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42 )

#scale features for processing
scaler = StandardScaler()
scale = scaler.fit(x_train)
x_train = scale.transform(x_train)
x_test = scale.transform(x_test)

In [25]:
#create logistic regression model
model = LogisticRegression()
model.fit(x_train, y_train)
pred = model.predict(x_test)

In [26]:
score = accuracy_score(y_test, pred)
score

0.7951219512195122

In [32]:
#create confusion matrix
confusion_matrix(y_test, pred)


array([[73, 29],
       [13, 90]], dtype=int64)

The confusion matrix shows the number of true positives, true negatives, false positives, and false negatives for a binary classification problem. Based on the matrix, we can make the following observations:

- There are a total of 205 instances in the dataset.
- Out of the 102 instances that are actually negative, the model correctly predicted 73 of them as negative (true negatives), but incorrectly predicted 29 of them as positive (false positives).
- Out of the 103 instances that are actually positive, the model correctly predicted 90 of them as positive (true positives), but incorrectly predicted 13 of them as negative (false negatives).

Overall, this indicates that the model performed reasonably well, with a high number of true positives and true negatives. However, it also made a moderate number of false positive and false negative errors, which could be further investigated to determine why the model made those errors and whether improvements can be made.

In [39]:
#Confufion matrix metrics
matrix = classification_report(y_test, pred)

print('Classification Report: \n', matrix)

Classification Report: 
               precision    recall  f1-score   support

           0       0.85      0.72      0.78       102
           1       0.76      0.87      0.81       103

    accuracy                           0.80       205
   macro avg       0.80      0.79      0.79       205
weighted avg       0.80      0.80      0.79       205



#### The report shows that...

...for class 0 (no disease), the precision is 0.85, the recall is 0.72, and the f1-score is 0.78. This means that out of all the instances that the model predicted as class 0, 85% of them were actually class 0. The model correctly identified 72% of all class 0 instances in the dataset, and the f1-score is 0.78, which is a balance between precision and recall.

Similarly, for class 1(disease), the precision is 0.76, the recall is 0.87, and the f1-score is 0.81. This means that out of all the instances that the model predicted as class 1, 76% of them were actually class 1. The model correctly identified 87% of all class 1 instances in the dataset, and the f1-score is 0.81, which is a balance between precision and recall.

The overall accuracy of the model is 0.80, which means that it correctly predicted the labels for 80% of the instances in the dataset. The macro average and weighted average f1-score are both 0.79, indicating that the model performs similarly across both classes, but is slightly biased towards class 1 due to the higher recall score.